In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import json
from tqdm import tqdm_notebook as tqdm
from scipy.stats import pearsonr
import pandas as pd
import numpy as np
import os
from BasicModel import BasicModel

In [12]:
filepath = './datasets'
processed_dataset_filepath = './processed_data'
gauge_path = filepath+'/gauge.csv'
radar_path = filepath+'/radar'
station_location_path = processed_dataset_filepath +'/station_location.json'

In [13]:
with open(station_location_path, 'r') as infile:
    station_location = json.load(infile)

In [ ]:
df_gauge = pd.read_csv(gauge_path)

In [ ]:
df_gauge_array = df_gauge.values[:, 2:]
print (df_gauge_array.shape)

In [ ]:
np.where(pd.isnull(df_gauge))

In [ ]:
all_radar_path_list = sorted(os.listdir(radar_path))

In [ ]:
df_gauge.head()
df_gauge['Week'] = df_gauge['Week'].astype(str)

In [ ]:
train_week = []
list_dfs = []
for filename in tqdm(all_radar_path_list):
    week = filename.split('.')[0][-2:]
    train_week.append(str(week))
    full_radar_path = os.path.join(radar_path, filename)
    df_radar = pd.read_csv(full_radar_path)
    df_radar['latlong'] = df_radar['lat'] * 1000 + df_radar['long']
    list_dfs.append(df_radar)

In [ ]:
df_result = pd.DataFrame(columns = column_names)
for station in tqdm(range(df_gauge_array.shape[1])):
    for week in train_week:
        chosen_latlong = int(station_location[str(station)]['latlong'])
        y = df_gauge[df_gauge['Week'] == str(week)][str(station)].values[0]
        week_index = train_week.index(week)
        df_radar = list_dfs[week_index]
        x = df_radar[df_radar['latlong'] == chosen_latlong].values[:, 2:36][0]
        df_row = pd.DataFrame([np.concatenate(([station], [week], x, [y]))], columns = column_names)
        df_result = pd.concat([df_result, df_row])      

In [4]:
import numpy as np
np.sqrt(10+2)

3.4641016151377544

In [ ]:
df_result.to_csv('./preprocessed_data/train.csv', index=False)

In [14]:
train_path = processed_dataset_filepath+'/train.csv'

In [17]:
df_result = pd.read_csv(train_path)
del df_result['Unnamed: 0']

In [18]:
df_result.head()

,station,week,0,1,2,3,4,5,6,7,...,25,26,27,28,29,30,31,32,33,y
0,0,13,1985,0,3,0,1,1,0,4,...,0,0,0,0,0,0,0,0,0,36.8
1,0,15,1993,0,0,1,0,0,2,0,...,0,0,0,0,0,0,0,0,0,43.8
2,0,16,1945,1,1,0,3,4,4,2,...,0,0,0,0,0,0,0,0,0,98.8
3,0,17,1976,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,100.2
4,0,18,2005,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,2.0


In [19]:
X_array = df_result.values[:,2:36].astype(float)
y_array = df_result.values[:, 36].astype(float)

In [81]:
basicmodel = BasicModel()

In [82]:
basicmodel.loss(0.079, 0.228, X_array, y_array)

522650849.0630083

In [83]:
basicmodel.predict(X_array)

array([ 1397.60762196,  1366.82310835,  3814.47871354, ...,
         567.18886523, 14440.36820761,  7640.9016897 ])

In [85]:
basicmodel.fit(X_array, y_array, learning_rate=0.0001, beta_1 = 0.90, beta_2 = 0.999 max_iter=10000)

Initial_loss: 4472.394264516129
Shape of x: (1550, 34)
Shape of y: (1550,)
A: 0.13654214687534183
B: 0.006225778467215442
Loss: 2554.5294574771947


In [ ]:
basicmodel.plot_lost()

In [86]:
basicmodel.predict(X_array)

array([22.96448081, 22.96039009, 23.00315037, ..., 22.94526716,
       23.00518832, 22.95955425])

In [57]:
y_array

array([ 36.8,  43.8,  98.8, ...,   2.4, 159. ,  34.8])